In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [3]:
loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [4]:
len(docs)

5

In [ ]:
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

e:\Development\langchain\langchain_models\lang_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
db=FAISS.from_documents(docs,embedding)
db

In [8]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs

[Document(id='db2d9926-106c-4165-8d5d-7c78091966e1', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='17e5cfb1-2d09-4f57-bb2b-8452b71fa748', metadata={'source': 'speech.txt'}, page_content='T

In [9]:
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

### Using vector store as retriever
Converting vector store db to retriever allows us to use vector store as runnable and use it with other langchain components

In [10]:
vs_retriever=db.as_retriever()

In [11]:
vs_retriever.invoke(query)

[Document(id='db2d9926-106c-4165-8d5d-7c78091966e1', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='17e5cfb1-2d09-4f57-bb2b-8452b71fa748', metadata={'source': 'speech.txt'}, page_content='T

#### Similarity Search with score
Allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [12]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='db2d9926-106c-4165-8d5d-7c78091966e1', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(1.3070924)),
 (Document(id='17e5cfb1-2d09-4f57-bb2b-8452b71fa748', metadata={'source': 's

### Similarity search by embedding vectors

In [13]:
embedding_vector=embedding.embed_query(query)
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='db2d9926-106c-4165-8d5d-7c78091966e1', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='17e5cfb1-2d09-4f57-bb2b-8452b71fa748', metadata={'source': 'speech.txt'}, page_content='T

In [14]:
### Saving And Loading
db.save_local("faiss_index")

In [16]:
new_db=FAISS.load_local("faiss_index",embedding,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)
docs

[Document(id='db2d9926-106c-4165-8d5d-7c78091966e1', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='17e5cfb1-2d09-4f57-bb2b-8452b71fa748', metadata={'source': 'speech.txt'}, page_content='T